In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [ ]:
def compute_loss(y, tx, w):
    return np.sum(np.power(y- tx @ w,2))/(2*len(y))

In [4]:
def least_squares(y, tx):
    weight = np.linalg.solve(tx.T @ tx,tx.T @ y)
    mse = compute_loss(y,tx,weight)
    return mse, weight

In [70]:
def compute_gradient(y, tx, w):
    e = y - tx @ w;
    return -(e @ tx)/len(tx);

def least_squares_GD(y, tx,initial_w, max_iters,gamma):
    weight = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: compute gradient and loss
        # ***************************************************
        gradient = compute_gradient(y,tx,w);
        loss = compute_loss(y,tx,w);
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: update w by gradient
        # ***************************************************
        w = w  - gamma*gradient;
        # store w and loss
        weight.append(np.copy(w))
        losses.append(loss)
    return losses, weight
    

In [71]:
mse,weight = least_squares(y,tX)
print(mse)
print(weight)
#m = np.linalg.lstsq(tX,y)
#print(m)
w_initial = np.zeros([tX.shape[1]])
max_iters = 50
gamma = 0.4
mse_GD,weight_GD = least_squares_GD(y,tX,w_initial,max_iters,gamma)
#print(mse_GD)
print(mse_GD)

0.339686809477
[  8.03494315e-05  -7.20202273e-03  -6.05417273e-03  -5.47559066e-04
  -1.93874702e-02   4.73451621e-04  -2.60379054e-02   3.25106300e-01
  -3.80780260e-05  -2.72729924e+00  -2.21220140e-01   9.50794092e-02
   6.40351616e-02   2.73555892e+00  -3.31801229e-04  -9.54325120e-04
   2.74031566e+00  -5.34164922e-04   9.73498609e-04   3.69225052e-03
   3.54487428e-04  -5.43344599e-04  -3.30448035e-01  -1.40800497e-03
   8.31432882e-04   1.02117272e-03  -1.68047416e-03  -5.83664815e-03
  -1.11087997e-02   2.72775917e+00]


/Users/Mick/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: overflow encountered in power
  from ipykernel import kernelapp as app


[0.5, 321307158192.89911, 1.6899972059798306e+24, 8.8923482315055436e+36, 4.6789410249316453e+49, 2.4619469188945865e+62, 1.2954176168484329e+75, 6.8161778353679628e+87, 3.5865098389191794e+100, 1.8871357431315748e+113, 9.9296571679781113e+125, 5.2247482372392352e+138, 2.7491376268827314e+151, 1.4465305022115323e+164, 7.6112977152074246e+176, 4.004882912662562e+189, 2.1072736534914978e+202, 1.1087970229189886e+215, 5.8342248810305117e+227, 3.0698296675461348e+240, 1.6152709879914475e+253, 8.4991698146315541e+265, 4.4720599871460633e+278, 2.3530910624004098e+291, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, nan]


## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)